# Edmonton Property Price Prediction Using FastAI
The machine learning workflow in this notebook is mostly pulled from the FastAI tabular models lesson.

In [2]:
from fastai.tabular import *

In [3]:
path = 'datasets/edmonton-property.csv'
df = pd.read_csv(path)

Need to specify processing, categorical names, continuous names, and the dependent variable.

In [4]:
df.head()

,Account Number,Suite,House Number,Street Name,Assessed Value,Assessment Class,Neighbourhood ID,Neighbourhood,Ward,Garage,Latitude,Longitude
0,8392904,NaN,9035.0,90 STREET NW,352500,Residential,6040.0,BONNIE DOON,Ward 8,Y,53.526362,-113.464935
1,10063906,NaN,1535.0,37B AVENUE NW,382500,Residential,6443.0,TAMARACK,Ward 11,Y,53.473516,-113.366455
2,4252557,NaN,8409.0,156 AVENUE NW,348000,Residential,2050.0,BELLE RIVE,Ward 3,Y,53.616207,-113.472303
3,10512253,NaN,7046.0,NEWSON ROAD NW,168000,Residential,3111.0,GRIESBACH,Ward 2,N,53.608989,-113.513399
4,8855603,NaN,7112.0,82 STREET NW,370500,Residential,6020.0,AVONMORE,Ward 11,Y,53.507129,-113.453902


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405973 entries, 0 to 405972
Data columns (total 12 columns):
Account Number      405973 non-null int64
Suite               80070 non-null object
House Number        362201 non-null float64
Street Name         362201 non-null object
Assessed Value      405973 non-null int64
Assessment Class    405973 non-null object
Neighbourhood ID    405972 non-null float64
Neighbourhood       405972 non-null object
Ward                405972 non-null object
Garage              405973 non-null object
Latitude            405973 non-null float64
Longitude           405973 non-null float64
dtypes: float64(4), int64(2), object(6)
memory usage: 37.2+ MB


## Process and Visualize Data

In [9]:
for col in df.columns:
    print(col, df[col].nunique())

Account Number 405973
Suite 1892
House Number 18570
Street Name 2886
Assessed Value 9140
Assessment Class 4
Neighbourhood ID 399
Neighbourhood 399
Ward 12
Garage 2
Latitude 261510
Longitude 261510


### 1. Account Number: Leave Off
Probably not going to be useful.

### 2. Suite: Use
This could be either categorical, or possibly numeric if higher suite numbers means higher floor? Maybe convert to a new column that just uses floor number?  Will need to do some processing and plotting to decide.

NOTE: Turns out there's a bunch of strings in the suite column.  Will probably need do some regex stuff if i want to use those ones.

In [11]:
# want to find most common suite numbers, notice non ints
df.Suite.value_counts().sort_values(ascending=False)

201      1054
202      1016
101       996
203       988
301       969
         ... 
14709       1
5203        1
2-305       1
5112        1
J1          1
Name: Suite, Length: 1892, dtype: int64

In [12]:
df.Suite.describe()

count     80070
unique     1892
top         201
freq       1054
Name: Suite, dtype: object

Definitely should plot the relationship between different suite number derivatives and assessed price.
- First digit vs price
- Full number vs price

In [45]:
full_suite = df[['Suite', 'Assessed Value']]

Need to get the suite dtype to be ints.  Stuck as objects of some kind right now.

In [46]:
full_suite.Suite = full_suite.Suite.apply(lambda x: pd.to_numeric(x, errors='coerce'))
full_suite.Suite = full_suite.Suite.fillna(1)

Apparently tries to convert to numeric, making any fails into NaN, then fills with 1

In [50]:
full_suite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405973 entries, 0 to 405972
Data columns (total 2 columns):
Suite             405973 non-null float64
Assessed Value    405973 non-null int64
dtypes: float64(1), int64(1)
memory usage: 6.2 MB


In [ ]:
dep_var = 'Assessed Value'
cat_names = ['Street Name', 'Assessment Class',  ]
cont_names = []
procs = [FillMissing, Categorify, Normalize]